In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
data = pd.read_csv('Churn_Modelling.csv')

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Processing the data

data = data.drop(['RowNumber','CustomerId', 'Surname'], axis=1)

In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [8]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [10]:

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [11]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
data=pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [13]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
## Split the data into independed and dependent

X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

##Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##Scaling the featuers
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
##Save the encoders and scaler

with open('label_encoder_gender_reg.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo_reg.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler_reg.pkl', 'wb') as file:
    pickle.dump(scaler, file)


## Train the ANN with regression problem statment :D

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)), #HL1 #You only need the input_shapre fore the first hidden layer
        Dense(32, activation='relu'), #HL 2
        Dense(1) ##Output layer : Note, the base activation function is linear regression
    ]
)

## Complite the model

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "reg/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [21]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [22]:
## Training the model

history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 100376.7188 - mae: 100376.7188 - val_loss: 98511.2266 - val_mae: 98511.2266
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99578.4219 - mae: 99578.4219 - val_loss: 96892.2656 - val_mae: 96892.2656
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 96723.0000 - mae: 96723.0000 - val_loss: 92719.0859 - val_mae: 92719.0859
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 91109.9531 - mae: 91109.9531 - val_loss: 85810.2656 - val_mae: 85810.2656
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 83032.8438 - mae: 83032.8438 - val_loss: 77022.4922 - val_mae: 77022.4922
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 73806.1172 - mae: 73806.1172 - val_loss: 68124.6094 - val_mae: 68124.6094
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 65062.5977 - mae: 65062.597

In [23]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir reg/logs/fit

In [25]:
## Evaluate the model on the test data

test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 0s 2ms/step - loss: 50386.1094 - mae: 50386.1094
Test MAE: 50386.109375


In [26]:
model.save('regression_model.h5')

c:\Users\Cameron\Documents\UDEMY-CODE\DLetoe\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
